# SQL Validation

## Objective

Validate key analytical insights derived in python by reproducing them using sql, ensuring consistency and strengthening business credibility


In [1]:
import numpy as np,pandas as pd
import sqlite3

### load Data


In [2]:
market_df = pd.read_csv(r'C:\Users\sande\OneDrive\Documents\Data science AI&ML\Projects\Projects for Resume\Stock Market Analysis & Forecasting- Project\Data\processed\market_data.csv')
market_df.head()

,date,reliance_open,reliance_high,reliance_low,reliance_close,reliance_volume,nifty_open,nifty_high,nifty_low,nifty_close,nifty_volume,reliance_return,nifty_return
0,2021-01-04,904.640696,906.363699,892.352703,902.713623,24513534,14104.34961,14147.95020,13953.75000,14132.90039,495000,0.001686,0.008161
1,2021-01-05,892.806130,899.426244,886.911511,891.491211,24123091,14075.15039,14215.59961,14048.15039,14199.50000,492500,-0.012432,0.004712
2,2021-01-06,891.400457,891.445793,863.854552,867.980774,46401468,14240.95020,14244.15039,14039.90039,14146.25000,632300,-0.026372,-0.003750
3,2021-01-07,870.814696,881.923778,863.854564,866.575134,32325918,14253.75000,14256.25000,14123.09961,14137.34961,559200,-0.001619,-0.000629
4,2021-01-08,869.681241,878.931255,867.006006,876.800110,27540187,14258.40039,14367.29980,14221.65039,14347.25000,613500,0.011799,0.014847


In [4]:
'''Create SQLite temporary database 
 open a connection to it 
store my DataFrame as a SQL table named market_data.'''

conn = sqlite3.connect(":memory:")
market_df.to_sql('market_data', conn, index =False, if_exists='replace')

1238

### table structure

In [5]:
pd.read_sql("PRAGMA table_info(market_data);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,date,TEXT,0,None,0
1,1,reliance_open,REAL,0,None,0
2,2,reliance_high,REAL,0,None,0
3,3,reliance_low,REAL,0,None,0
4,4,reliance_close,REAL,0,None,0
5,5,reliance_volume,INTEGER,0,None,0
6,6,nifty_open,REAL,0,None,0
7,7,nifty_high,REAL,0,None,0
8,8,nifty_low,REAL,0,None,0
9,9,nifty_close,REAL,0,None,0


SQLite stores dates as text by design. Since the format is ISO-standard, it works correctly with date functions like strftime

### Sql query1 ,Average Daily returns


In [6]:
pd.read_sql("""
SELECT 
            avg(reliance_return) as avg_reliance_return,
            avg(nifty_return) as avg_nifty_return
FROM market_data;""", conn)

,avg_reliance_return,avg_nifty_return
0,0.000556,0.000545


*  Matches Python means (~0.000556 vs ~0.000545).
* This confirmed that reliance has slightly higher average daily returns

### Monthly avarage returns

In [7]:
pd.read_sql("""
Select 
            strftime('%Y-%m', date) as month,
            avg(reliance_return) as avg_reliance_return,
            avg(nifty_return) as avg_nifty_return
            
FROM market_data
group by month
order by month;""", conn)

,month,avg_reliance_return,avg_nifty_return
0,2021-01,-0.003770,-0.001408
1,2021-02,0.006404,0.003320
2,2021-03,-0.001760,0.000601
3,2021-04,-0.000129,-0.000131
4,2021-05,0.004149,0.003188
...,...,...,...
56,2025-09,0.000246,0.000351
57,2025-10,0.004163,0.002116
58,2025-11,0.002837,0.000986
59,2025-12,0.000112,-0.000117


### Volatality Comparison

In [17]:
pd.read_sql("""
SELECT
            sqrt(AVG(reliance_return * reliance_return)) as reliance_volality,
            sqrt(avg(nifty_return * nifty_return)) as nifty_volatality
from market_data;""", conn)




,reliance_volality,nifty_volatality
0,0.014313,0.008817


SQL validation confirmed that Reliance carries significantly higher volatility than the benchmark, reinforcing the risk–return trade-off.

### Best & worst months

In [26]:
pd.read_sql("""
SELECT 
            strftime('%Y-%m', date) as best_month,
            avg(reliance_return) as avg_reliance_return
FROM market_data
group by best_month
order by avg_reliance_return DESC
limit 1;""", conn)



,best_month,avg_reliance_return
0,2021-02,0.006404


In [25]:
pd.read_sql("""
SELECT 
            strftime('%Y-%m', date) as worst_month,
            avg(reliance_return) as avg_reliance_return
from market_data
group by worst_month
order by avg_reliance_return ASC
limit 1;""", conn)

,worst_month,avg_reliance_return
0,2022-09,-0.004609


### Volume & volatility relationship 

In [30]:
pd.read_sql(""" 
SELECT 
            strftime('%Y-%m', date) as month,
            avg(reliance_volume) as avg_volume,
            avg(abs(reliance_return)) as avg_abs_return
from market_data
group by month
order by avg_volume DESC;""", conn
)

,month,avg_volume,avg_abs_return
0,2021-01,3.369205e+07,0.018228
1,2021-02,2.612424e+07,0.014836
2,2021-06,2.288900e+07,0.009967
3,2022-07,2.032833e+07,0.012937
4,2021-03,2.003128e+07,0.014116
...,...,...,...
56,2023-05,9.934956e+06,0.006984
57,2025-09,9.705384e+06,0.005166
58,2022-12,8.993065e+06,0.007592
59,2025-12,8.344592e+06,0.005732


Higher trading volume coincided with higher absolute returns, indicating stronger market participation during volatile periods.

### Final

SQL validation confirmed the same performance and risk patterns observed in Python.
Reliance exhibits higher returns during positive market phases but experiences
greater downside and volatility during weak periods. Volume analysis further
supported periods of market stress.
